#Emotions Recognition AI Project

### 사용할 모델 다운로드
#### face detection adas 0001 다운로드

In [1]:
! omz_downloader --name face-detection-adas-0001 --precision FP16

################|| Downloading face-detection-adas-0001 ||################

========== Downloading C:\BrainAI\emotion-recognition\intel\face-detection-adas-0001\FP16\face-detection-adas-0001.xml
... 100%, 304 KB, 330 KB/s, 0 seconds passed

========== Downloading C:\BrainAI\emotion-recognition\intel\face-detection-adas-0001\FP16\face-detection-adas-0001.bin
... 49%, 1024 KB, 170 KB/s, 6 seconds passed
... 99%, 2048 KB, 277 KB/s, 7 seconds passed
... 100%, 2056 KB, 278 KB/s, 7 seconds passed



In [2]:
! omz_downloader --name emotions-recognition-retail-0003 --precision FP16

################|| Downloading emotions-recognition-retail-0003 ||################

========== Downloading C:\BrainAI\emotion-recognition\intel\emotions-recognition-retail-0003\FP16\emotions-recognition-retail-0003.xml
... 100%, 54 KB, 55 KB/s, 0 seconds passed

========== Downloading C:\BrainAI\emotion-recognition\intel\emotions-recognition-retail-0003\FP16\emotions-recognition-retail-0003.bin
... 21%, 1024 KB, 744 KB/s, 1 seconds passed
... 42%, 2048 KB, 1272 KB/s, 1 seconds passed
... 63%, 3072 KB, 1890 KB/s, 1 seconds passed
... 84%, 4096 KB, 2240 KB/s, 1 seconds passed
... 100%, 4848 KB, 2315 KB/s, 2 seconds passed



## 필요한 라이브러리 Import

In [3]:
import openvino as ov
import cv2
import numpy as np

import matplotlib.pyplot as plt
from pathlib import Path

### inference 할 device 확인

In [4]:
core = ov.Core()
options = core.available_devices

options

['CPU', 'GPU']

In [5]:
core = ov.Core()

model = core.read_model(model='./models/face-detection-adas-0001.xml')
face_model = core.compile_model(model=model, device_name="CPU")

face_input_layer = face_model.input(0)
face_output_layer = face_model.output(0)

print("Input layer shape: ", face_input_layer.shape)
print("Output layer shape:", face_output_layer.shape)

Input layer shape:  [1,3,384,672]
Output layer shape: [1,1,200,7]


In [6]:
core = ov.Core()

model = core.read_model(model='./models/emotions-recognition-retail-0003.xml')
emotion_model = core.compile_model(model=model, device_name="CPU")

emotion_input_layer = emotion_model.input(0)
emotion_output_layer = emotion_model.output(0)

print("Input layer shape: ", emotion_input_layer.shape)
print("Output layer shape:", emotion_output_layer.shape)

Input layer shape:  [1,3,64,64]
Output layer shape: [1,5,1,1]


## Load Image

In [20]:
frame = cv2.imread("images/emotions.jpg")

resized_frame = cv2.resize(src=frame, dsize=(672, 384)) 
transposed_frame = resized_frame.transpose(2, 0, 1)
input_frame = np.expand_dims(transposed_frame, 0)

In [21]:
def DrawBoundingBoxes(output, frame, conf=0.5):
    boxes = []
    canvas = frame.copy()
    h,w,_ = canvas.shape 

    predictions = output[0][0]            # 하위 집합 데이터 프레임
    confidence = predictions[:,2]         # conf 값 가져오기 [image_id, label, conf, x_min, y_min, x_max, y_max]

    top_predictions = predictions[(confidence>conf)]         # 임계값보다 큰 conf 값을 가진 예측만 선택

    for detection in top_predictions:
        box = (detection[3:7] * np.array([w, h, w, h])).astype("int") # 상자 위치 결정
        (xmin, ymin, xmax, ymax) = box   # xmin, ymin, xmax, ymax에 상자 위치 값 지정
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)       # 사각형 그리기
        boxes.append(box)     #이미지에 박스를 그린 얼굴의 위치 저장
   
    return boxes

In [34]:
def DrawText(output, frame, face_position):
    # emotions 딕셔너리 생성
    emotions = {
        0:"neutral",
        1:"happy",
        2:"sad",
        3:"surprise",
        4:"anger"
    }
    # 딕셔너리 출력하기
    #for key, value in emotions.items():
    #    print(key, value, end='      ')
    #print()
        
    predictions = output[0,:,0,0]              # 5개의 감정 예측값 저장
    print("predictions : " + str(predictions))
    
    topresult_index = np.argmax(predictions)   # 5개의 감정 예측값 중 가장 높은값의 위치 저장
    #print("topresult_index : " + str(topresult_index))
    
    emotion = emotions[topresult_index]        # emotions에서 topresult_index 값에 해당하는 감정 저장
    #print("emotion : " + emotion)

    cv2.putText(frame, emotion,                 # 예측한 감정값 이미지에 출력하기
                (face_position[0],face_position[1]),    #xmin, ymin 값을 가져와 위치 설정
                cv2.FONT_HERSHEY_SIMPLEX, 1, 
                (0, 0,255), 2)


In [35]:
face_output = face_model([input_frame])[face_output_layer]
boxes = DrawBoundingBoxes(face_output, frame, conf=0.5)

In [36]:
if boxes is not None:
    
    for box in boxes:          #boxes에 저장된 얼굴의 위치들을 하나씩 box에 전달
    
        xmin, ymin, xmax, ymax = box      #box에 저장된 좌표 저장
        emotion_input = frame[ymin:ymax,xmin:xmax]         #이미지에서 해당 얼굴 위치를 찾아 저장
        
        
        # 감정 인식 모델을 사용하기 위해 이미지 전처리
        # Input layer shape:  [1,3,64,64]
        resized_image = cv2.resize(src=emotion_input, dsize=(64, 64))      #이미지 사이즈 변경  (64,64,3)
        transposed_image = resized_image.transpose(2, 0, 1)                #shape 위치 변경    (3,64,64)
        input_image = np.expand_dims(transposed_image, 0)                  #차원 확장 (1,3,64,64)

        emotion_output = emotion_model([input_image])[emotion_output_layer]  # 감정 추론
        DrawText(emotion_output, frame, box)   # 추론의 결과값 이미지에 출력하기

predictions : [0.472979   0.00922723 0.03208937 0.12683505 0.35886937]
predictions : [1.8001714e-04 1.5403769e-03 9.2567647e-01 2.4386555e-04 7.2359294e-02]
predictions : [5.2245511e-03 7.2852528e-04 8.5551362e-04 9.9174678e-01 1.4446168e-03]
predictions : [1.6682513e-01 8.2122630e-01 9.5314812e-03 1.8027073e-04 2.2369029e-03]
predictions : [3.9358638e-04 9.7996461e-01 1.5637850e-02 2.5980277e-03 1.4059981e-03]
predictions : [5.8812648e-04 5.6197602e-02 1.8172228e-04 9.1954255e-01 2.3489922e-02]
predictions : [6.7086075e-05 9.9930334e-01 7.4545591e-05 4.5859793e-04 9.6366646e-05]
predictions : [5.3750403e-04 1.4339936e-04 1.6935866e-02 4.8260146e-05 9.8233497e-01]
predictions : [0.0036957  0.9840232  0.00687442 0.00285215 0.00255446]
predictions : [1.0526952e-01 3.5744105e-04 7.9664819e-02 1.4745106e-04 8.1456077e-01]
predictions : [4.2153178e-03 8.6598091e-02 1.0233697e-04 8.9234114e-01 1.6743194e-02]
predictions : [1.6983333e-03 5.4251018e-04 3.2775700e-03 1.5957872e-05 9.9446565e-01

In [37]:
cv2.imshow("emotion-recognition", frame)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [38]:
def AddBackground(frame, bg):

    frame_h, frame_w = frame.shape[0], frame.shape[1]
    new_h = 500
    new_w = int((new_h/frame_h)*frame_w)
    frame_resize = cv2.resize(frame, (new_w, new_h))

    xmax = bg.shape[1] - 400
    ymax = bg.shape[0] - 175
    xmin = xmax - new_w
    ymin = ymax - new_h

    bg[ymin:ymax, xmin:xmax] = frame_resize

    return bg

In [39]:
background = "./images/Background2.jpg"  #사용할 배경화면 경로

bg = cv2.imread(background)
deployment = AddBackground(frame, bg)

In [40]:
cv2.imshow("Deployment", deployment)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
camera = cv2.VideoCapture(0) #create a VideoCapture object with the 'first' camera (your webcam)
background = "./images/Background2.jpg"
bg = cv2.imread(background)

while(True):
    ret, frame = camera.read()             # Capture frame by frame      
    if ret == False:
        break
    
    resized_frame = cv2.resize(src=frame, dsize=(672, 384)) 
    transposed_frame = resized_frame.transpose(2, 0, 1)
    input_frame = np.expand_dims(transposed_frame, 0)    
    
    face_output = face_model([input_frame])[face_output_layer]
    
    boxes = DrawBoundingBoxes(face_output, frame, conf=0.5)
    
    if boxes is not None:
    
        for box in boxes:          #boxes에 저장된 얼굴의 위치들을 하나씩 box에 전달
    
            xmin, ymin, xmax, ymax = box      #box에 저장된 좌표 저장
            emotion_input = frame[ymin:ymax,xmin:xmax]         #이미지에서 해당 얼굴 위치를 찾아 저장
        
            # 감정 인식 모델을 사용하기 위해 이미지 전처리
            # Input layer shape:  [1,3,64,64]
            resized_image = cv2.resize(src=emotion_input, dsize=(64, 64))      #이미지 사이즈 변경  (64,64,3)
            transposed_image = resized_image.transpose(2, 0, 1)                #shape 위치 변경    (3,64,64)
            input_image = np.expand_dims(transposed_image, 0)                  #차원 확장 (1,3,64,64)

            emotion_output = emotion_model([input_image])[emotion_output_layer]  # 감정 추론
            DrawText(emotion_output, frame, box)   # 추론의 결과값 이미지에 출력하기
    
    deployment = AddBackground(frame, bg)
    
    cv2.imshow('Press Spacebar to Exit', deployment)

    if cv2.waitKey(1) & 0xFF == ord(' '):  # Stop if spacebar is detected
        break

camera.release()                           # Cleanup after spacebar is detected.
cv2.destroyAllWindows()

predictions : [0.6575115  0.01411181 0.2312289  0.01306061 0.08408716]
predictions : [0.66122824 0.00940636 0.19746596 0.06479784 0.0671016 ]
predictions : [0.6292789  0.00792609 0.2539939  0.05373932 0.05506181]
predictions : [0.45382342 0.00962801 0.39550188 0.11329404 0.02775266]
predictions : [0.48729637 0.01030625 0.4251632  0.04088591 0.03634826]
predictions : [0.548343   0.00959724 0.385616   0.0257691  0.03067462]
predictions : [0.61153275 0.01663362 0.30016324 0.02998387 0.04168652]
predictions : [0.5361002  0.01008574 0.38847083 0.01488427 0.05045893]
predictions : [0.5229921  0.01256903 0.39606693 0.01656517 0.05180679]
predictions : [0.60425365 0.01813449 0.3114331  0.01793112 0.04824767]
predictions : [0.66643053 0.01971855 0.24182945 0.01707554 0.05494598]
predictions : [0.6990299  0.02584182 0.18138845 0.01747223 0.07626753]
predictions : [0.6873923  0.0310193  0.1973717  0.01532985 0.06888693]
predictions : [0.6477885  0.05574694 0.18167506 0.01093538 0.10385409]
predic